# 6. Osztályok, filekezelés
_2021.10.14_

Tartalom:
- Tipikus hibák
- Osztályok
- Package/module készítése
- Fájlkezelés (alapértelmezett)
- Hibakezelés
- Datetime

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

## 6.1 Tipikus hibák
- Nem célszerű `reserved word` változóneveket használni, pl: list, filter stb.
- A tanult sorbarendező eljárások csak listák sorbarendezésére alkalmasak, objektumok sorbarendezésére nem
- Jupyter notebook használatábal célszerű szétszedni az alkalmazást, így az egyes elemek külön is futtathatóak
- Nem ajánlatos nemzetiségi karaktereket használni a változók és függvények nevében
- A függvények és változók neve legyen beszédes (pl: ne fuggveny1, hanem file_beolvas)
- Néha érdemes a kernelt resetelni, így biztosítható, hogy a kód fut

## 6.2 Osztályok, öröklődés
Nagyobb alkalmazások fejlesztése során célszerű az egybe tartozó részeket osztályokba szervezni.
### 6.2.1 Scope, namespaces
A programunk logikailag egymáshoz tartozó részét egy __namespace__ (névtér alá) célszerű elhelyezni. A __scope__ (magyarul láthatóság) a változók és függvények láthatóságára utal. A függvényen vagy osztályon kívül definiált változóknak és függvényeknek globális a láthatóságuk. A __global__ kulcsszóval tudjuk elérni a globális változókat függvényen belülről. A __nonlocal__ kulcsszóval el tudjuk értni a nem globális és nem lokális változókat.

In [2]:
def scope_test():
    def do_local():
        spam = "local spam"

    def do_nonlocal():
        nonlocal spam
        spam = "nonlocal spam"

    def do_global():
        global spam
        spam = "global spam"

    spam = "test spam"
    do_local()
    print("After local assignment:", spam)
    do_nonlocal()
    print("After nonlocal assignment:", spam)
    do_global()
    print("After global assignment:", spam)

scope_test()
print("In global scope:", spam)

After local assignment: test spam
After nonlocal assignment: nonlocal spam
After global assignment: nonlocal spam
In global scope: global spam


### 6.2.2 Classes
Az osztályok fogják össze a funkcionálisan egybe tartozó változókat és függvényeket. Általánosságban az osztályok példányosíthatók és más osztályok származtathatók belőlük. Az osztályhoz tartozó függvények első paramétere _self_ kell legyen, ugyanis ezen keresztül érhető el a példányosított osztály változói és függvényei.

In [3]:
class MyClass:
    my_var = 12345

    def my_func(self):
        return 'test'

In [10]:
# a MyClass osztály példányosítása
# az x változó a MyClass egy példányára mutat
x = MyClass()

__init__ function called


In [5]:
# osztályváltozó elérése
x.my_var

12345

In [6]:
x.my_func()

'test'

### 6.2.3 Instantiation
A példányosítás során az osztályról készítünk egy másolatot, amelyet eltérő adatokkal tölthetünk fel.

In [7]:
class MyClass:
    my_var = 12345
    def __init__(self):
        print('__init__ function called')
        self.my_var = 54321

    def my_func(self):
        return 'test'

In [8]:
x = MyClass()

__init__ function called


In [9]:
x.my_var

54321

### 6.2.4 Osztály & példány változók
Az osztályváltozók az osztályhoz köthető, a példányváltozók pedig a példányokból érhetők el

In [1]:
class Kutya:
    fajta = 'kutya' # osztályváltozó

    def __init__(self, nev):
        self.nev = nev    # példányváltozó

In [2]:
d = Kutya('Buksi')
e = Kutya('Blöki')

In [3]:
d.fajta, d.nev

('kutya', 'Buksi')

In [5]:
# a fajta ugyanaz marad, a név változik
e.fajta, e.nev

('kutya', 'Blöki')

> **1. feladat**: Egészítsd ki a Kutya osztályt egy tanul_trukk függvénnyel, amely a paraméterként átadott trükköt hozzáadja az adott kutya megtanult trükkjei közé!

Rossz megoldás

In [6]:
class Kutya:
    fajta = 'kutya' # osztályváltozó
    trukkok = []

    def __init__(self, nev):
        self.nev = nev    # példányváltozó

    def tanul_trukk(self, trukk):
        self.trukkok.append(trukk)

In [7]:
d = Kutya('Fido')
e = Kutya('Mancika')
d.tanul_trukk('ül')
d.tanul_trukk('forog')

In [8]:
d.trukkok

['ül', 'forog']

In [9]:
e.trukkok

['ül', 'forog']

Jó megoldás

In [10]:
# A trukkok listát a Kutya osztály konstruktorában hozzuk létre példányváltozóként
class Kutya:
    fajta = 'kutya' # osztályváltozó

    def __init__(self, nev):
        self.nev = nev    # példányváltozó
        self.trukkok = []

    def tanul_trukk(self, trukk):
        self.trukkok.append(trukk)

In [11]:
d = Kutya('Fido')
e = Kutya('Mancika')
d.tanul_trukk('ül')
d.tanul_trukk('forog')

In [12]:
d.trukkok

['ül', 'forog']

In [13]:
e.trukkok

[]

### 6.2.5 Öröklődés
Az osztályokból más osztályok származtathatóak, amelyek öröklik az ősosztály változóit és függvényeit

In [14]:
# Ősosztály
class Jarmu:
    def __init__(self, szin):
        self.szin = szin

    def megjelenit(self):
        print('A jármű színe: {:}'.format(self.szin))

In [15]:
j = Jarmu('piros')
j.megjelenit()

A jármű színe: piros


In [16]:
# Az Auto osztályt a Jarmu osztályból származtatjuk
class Auto(Jarmu):
    def __init__(self, szin, kerekek_szama):
        # Az ősosztály konstruktorának meghívása
        Jarmu.__init__(self, szin)
        #super().__init__(szin)
        
        self.kerekek_szama = kerekek_szama

    # a megjelenit függvényt felülírjuk
    def megjelenit(self):
        print('A jármű színe: {:}\nKerekeinek száma: {:}'.format(self.szin, self.kerekek_szama))

class Repulo(Jarmu):
    pass

In [17]:
a = Auto('kek', 4)

In [18]:
a.megjelenit()

A jármű színe: kek
Kerekeinek száma: 4


> **2. feladat**: Fejezd be a Repulo osztályt, paraméterként adj hozzá egy Boolean típusú változót, amely azt tárolja, hogy a repülőgép futóműve behúzható-e!

In [19]:
class Repulo(Jarmu):
    def __init__(self, szin, behuzhato):
        super().__init__(szin)
        
        self.behuzhato = behuzhato

    def megjelenit(self):
        print('A jármű színe: {:}\nFutómű behúzható: {:}'.format(self.szin, self.behuzhato))

In [20]:
r = Repulo('zöld', False)
r.megjelenit()

A jármű színe: zöld
Futómű behúzható: False


## 6.3 Filekezelés
Olvassuk be a 6.2_airtravel.csv file tartalmát, amely a havi transzatlanti polgári repülések számát tartalmazza. Az egyes számok ezer utast jelentenek. A file-ok megnyitására az **open(filenév, mód)** parancs használható, ahol a
- _filenév_: a megnyitni kívánt file neve
- _mód_:
    - r: reading, megnyitás olvasásra (alapértelmezett)
    - w: writing: megnyitás írásra
    - a: append: megnyitás hozzáfűzésre
    - r+: megnyitás olvasásra és írásra

In [21]:
file = open('6.2_airtravel.csv', 'r')

In [22]:
print (file.read())

"Month", "1958", "1959", "1960"
"JAN",  340,  360,  417
"FEB",  318,  342,  391
"MAR",  362,  406,  419
"APR",  348,  396,  461
"MAY",  363,  420,  472
"JUN",  435,  472,  535
"JUL",  491,  548,  622
"AUG",  505,  559,  606
"SEP",  404,  463,  508
"OCT",  359,  407,  461
"NOV",  310,  362,  390
"DEC",  337,  405,  432




In [23]:
for each in file:
    print (each)

In [24]:
file = open('test.txt', 'w', encoding='utf-8')
file.write("Szöveg írása fileba")
file.close()

In [25]:
file = open('test.txt', 'a', encoding='utf-8')
file.write("\nMég egy sor")
file.close()

A with parancs használatával nem szükséges lezárni a file-t, mivel az automatikusan lezáródik.

In [26]:
with open("test.txt", encoding = 'utf-8') as file:  
    data = file.read()

In [27]:
data

'Szöveg írása fileba\nMég egy sor'

In [28]:
f = open("test.txt", encoding = 'utf-8')

In [29]:
f.read(6)

'Szöveg'

In [30]:
f.read(7)

' írása '

In [31]:
f.tell()

16

In [32]:
f.read()

'fileba\nMég egy sor'

In [33]:
f.tell()

36

In [34]:
f.seek(0)

0

In [35]:
f.read(13)

'Szöveg írása '

In [36]:
f.seek(0)

0

In [37]:
f.readline()

'Szöveg írása fileba\n'

In [38]:
f.readline()

'Még egy sor'

In [39]:
f.seek(0)

0

In [40]:
f.readlines()

['Szöveg írása fileba\n', 'Még egy sor']

## 6.4 Hibakezelés

In [41]:
10/0

ZeroDivisionError: division by zero

In [42]:
try:
    print(10/0)
except ZeroDivisionError:
    print("Véletlenül 0-val osztottunk")

Véletlenül 0-val osztottunk


In [43]:
raise NameError('Hello')

NameError: Hello

In [44]:
def divide(x, y):
    try:
        result = x / y
    except ZeroDivisionError:
        print("Nullával nem lehet osztani!")
    else:
        print("Az eredmény: ", result)
    finally:
        print("Finally")

In [45]:
divide(2, 1)

Az eredmény:  2.0
Finally


In [46]:
divide(2, 0)

Nullával nem lehet osztani!
Finally


In [47]:
divide("2", "1")

Finally


TypeError: unsupported operand type(s) for /: 'str' and 'str'

> **3. feladat**: Egészítsd ki a divide függvényt, hogy helytelen típusú input esetén is dobjon kivételt!

In [48]:
def divide(x, y):
    try:
        result = x / y
    except TypeError:
        print("Az input paraméterek típusa vagy float vagy integer kell legyen.")
    except ZeroDivisionError:
        print("Nullával nem lehet osztani!")
    else:
        print("Az eredmény: ", result)
    finally:
        print("Finally")

In [49]:
divide("2", "1")

Az input paraméterek típusa vagy float vagy integer kell legyen.
Finally


## 6.5 Dátumkezelés

### 6.5.1 Date
A dátumkezelő függvények használata esetén a date-t kell importálni a datetime module-ból.

In [50]:
from datetime import date

In [52]:
date.today()

datetime.date(2021, 10, 16)

In [53]:
a = date(2019, 4, 13)
print(a)

2019-04-13


### 6.5.2 Time
Az időkezelő függvények használata esetén a datetime-t kell importálni a datetime module-ból.

In [59]:
from datetime import datetime

In [60]:
a = datetime.now().time()
print(a)

10:36:01.632524


In [61]:
a.hour, a.minute, a.second, a.microsecond

(10, 36, 1, 632524)

### 6.5.3 Datetime

In [62]:
import datetime

x = datetime.datetime.now()

In [63]:
x

datetime.datetime(2021, 10, 16, 10, 36, 19, 619533)

In [64]:
x.year

2021

In [65]:
x.strftime("%A")

'Saturday'

In [66]:
x.strftime("%Y-%m-%d %H:%M:%S")

'2021-10-16 10:36:19'

In [95]:
date_string = "21 June, 2021"
datetime.strptime(date_string, "%d %B, %Y")

datetime.datetime(2018, 6, 21, 0, 0)

In [97]:
date_string = "2021-10-16"
datetime.strptime(date_string, "%Y-%m-%d")

datetime.datetime(2021, 10, 16, 0, 0)

### 6.5.4 Timestamp
A dátumot és az időt időbélyegként is eltárolhatjuk, amely float típusú és időzónát nem tárol el. A datetime és time típusú adatformátumok az időzónát is eltárolják.
[epochconverter.com](epochconverter)

In [68]:
import time

print(time.time())

1634373870.2188792


In [ ]:
timestamp = 1634142950
datetime.datetime.fromtimestamp(timestamp)

### 6.5.5 Timedelta
Dátumokkal való műveletek elvégézése esetén hasznos.

In [70]:
from datetime import datetime, date, timedelta

In [71]:
t1 = date(year = 2021, month = 7, day = 12)
t2 = date(year = 2020, month = 12, day = 23)
t3 = t1 - t2

In [72]:
t3, t3.days

(datetime.timedelta(days=201), 201)

In [73]:
t1 = datetime.now()
t2 = timedelta(days = 14)
t3 = t1 - t2

In [74]:
t3

datetime.datetime(2021, 10, 2, 10, 46, 59, 146879)

In [75]:
t2.total_seconds()

1209600.0

## 6.6 Package
[Python Modules and Packages – An Introduction](https://realpython.com/python-modules-packages/)

A moduláris programozás egy nagy, nehézkes programozási feladatot különálló, kisebb, jobban kezelhető részfeladatokká vagy modulokra való bontásának folyamatát jelenti.

Számos előnye van a modulok használatának az alkalmazásfejlesztés során:
- __Egyszerűség__: Ahelyett, hogy a teljes problémára összpontosítana, egy modul jellemzően a probléma egy viszonylag kis részére összpontosít, így kisebb a hibalehetőség.
- __Karbantarthatóság__: A modulokat általában úgy tervezték, hogy logikai határokat érvényesítsenek a különböző problématerületek között. Ha a modulok úgy íródnak, hogy minimálisra csökkentsék az egymásrautaltságot, csökken annak valószínűsége, hogy egyetlen modul módosításai hatással lesznek a program más részeire.
- __Láthatóság/scoping__: A modulok általában külön névteret határoznak meg, amely segít elkerülni a névütközést.
- __Újrafelhasználhatóság__: Az egyetlen modulban meghatározott funkcionalitást az alkalmazás más részei könnyen újra felhasználhatják (egy megfelelően meghatározott interfészen keresztül). Ez kiküszöböli a kód ismétlődésének szükségességét.

Pythonban az alábbi módokon lehetséges modulokat írni:
- __Python nyelven__
- __C nyelven__ megírni, majd futásidőben betölteni (pl: regex modul)
- __Beépített__ modulok, amelyeket az interpreter értelmez (pl: itertools)

Mindhárom esetben az import paranccsal importáljuk a modelokat.

Ha saját modult szeretnénk írni nincs másra szükségünk, mint egy .py kiterjesztésű file-ra, amely érvényes Python kódot tartalmaz.

In [ ]:
# module.py
a = "Not great, not terrible"
b = [1, 2, 3]

def c(p):
    print('A kapott paraméter: {:}'.format(p))

class d:
    pass

In [76]:
import module

In [79]:
module.a, module.b

('Not great, not terrible', [1, 2, 3])

In [80]:
module.c('hello')

A kapott paraméter: hello


In [81]:
module.d

module.d

Az interpreter a betöltendő modulokat az alábbi sorrendben keresi:
- az aktuális könyvtárban
- A PYTHONPATH környezeti változóban megadott könyvtárakban (import os;os.getenv('PYTHONPATH'))
- A Python telepítésekor beállított könyvtárak (import sys;sys.path)

In [87]:
# a module-t tartalmazó állomány helye
module.__file__

'C:\\Zol\\BCE\\Python\\module.py'

In [88]:
import re
re.__file__

'c:\\python39\\lib\\re.py'

## 6.6.1 import utasítás
Egy vagy több modul importálása:
```
import <osztály neve>[, <osztály neve> ...]
```
Egy modulból egy objektum importálása:
```
from <modul neve> import <név>
```
Egy modul importálása más néven:
```
import <modul neve> as <név>
```
Egy objektum importálása egy modulból más néven:
```
from <modul neve> import <név> as <név>
```
Egy modul objektumainak listázása:
```
dir(modul)
```

In [89]:
dir(module)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__spec__',
 'a',
 'b',
 'c',
 'd']

A **__name__** változó tartalmazza az aktuális modul nevét vagy **\_\_main\_\_**, ha nem modulként fut.

In [ ]:
# fact.py tartalma
# Futtatható standaloneként és modulként is.
def fact(n):
    return 1 if n == 1 else n * fact(n-1)

if (__name__ == '__main__'):
    import sys
    if len(sys.argv) > 1:
        print(fact(int(sys.argv[1])))

In [ ]:
from fact import fact
fact(6)

> **4. feladat**: Írj egy modult, amely egy `Datumkezelo` osztályt tartalmaz. Az osztály konstruktorának 1 string paramétere van. A dátumot yyyy-mm-dd formátumban kell megadni, ha nem így érkezik, akkor dobjon hibát. Legyen egy `megjelenit` osztály, amely kiírja a példányhoz tartozó dátumot és legyen egy `hozzaad_nap` függvény, amely paraméterként kapott napok számát hozzáadja a példányváltozóhoz. Minden függvényhez készíts hibakezelést, példányosítsd az osztályt, add át neki a mai napot a kívánt formátumban, irasd ki példányváltozó tartalmát, adj hozzá 5 napot és ismételten irasd ki a példányváltozó tartalmát.

In [114]:
class Datumkezelo():
    def __init__(self, date_string):
        try:
            self.date = datetime.strptime(date_string, "%Y-%m-%d")
        except ValueError:
            print("A dátumot yyyy-mm-dd formátumban kell megadni.")

    def megjelenit(self):
        print('Dátum: {:}'.format(self.date))

    def hozzaad_nap(self, nap):
        try:
            self.date = self.date + timedelta(days = nap)
        except TypeError:
            print('A megadott napok száma egész típusú kell legyen')

In [115]:
e = Datumkezelo('21-10-16')

A dátumot yyyy-mm-dd formátumban kell megadni.


In [118]:
d = Datumkezelo(datetime.now().strftime('%Y-%m-%d'))
d.megjelenit()
d.hozzaad_nap('5')
d.hozzaad_nap(4)
d.megjelenit()

Dátum: 2021-10-16 00:00:00
A megadott napok száma egész típusú kell legyen
Dátum: 2021-10-20 00:00:00


> **5. feladat**: Készíts egy `Person` osztályt, amely a konstruktorában átadott `name` változót betölti egy példányváltozóba. Származtass belőle egy `Student` osztályt, amelynek legyen egy `add_grade` függvénye, amely a paraméterként kapott jegyet hozzáadja egy példányváltozó listához. Készíts egy `mean` függvényt, amely a beírt jegyekből átlagot számol. Kezeld le azt az esetet, amikor a `mean` függvényt a jegyek beírása előtt hívjuk meg!